In [15]:
import os
import librosa, librosa.display
import matplotlib.pyplot as plt
from pathlib import Path
import tensorflow as tf
import IPython.display as ipd
import numpy as np
from scipy.fftpack import fft
import shutil
from pydub import AudioSegment

from functools import partial
#import ftransc

In [16]:
def load_audio(file_path):
    waveform, sample_rate = librosa.load(file_path)
    return waveform, sample_rate

def load_audio_files(path: str, label:str):

    dataset = []
    walker = sorted(str(p) for p in Path(path).glob(f'*.wav'))
    for i, file_path in enumerate(walker):
        path, filename = os.path.split(file_path)
        speaker, _ = os.path.splitext(filename)
        # speaker_id,version,gender,age,reason = speaker.split("-")
        #sp = speaker.split("-")
        #speaker_id = sp[0]
        #gender = sp[2]
        #age =sp[3]


        # Load audio
        waveform, sample_rate = load_audio(file_path)
        audio_segment = AudioSegment.from_file(file_path)
        dataset.append([waveform, sample_rate,audio_segment])
        #dataset.append([waveform, sample_rate, label, speaker_id, gender,age])

    return dataset



In [17]:
#dustan_baby
burping_data = load_audio_files('./data/sound_data/dustan_baby/burpme/', 'burpme')
tired_data = load_audio_files('./data/sound_data/dustan_baby/tired_owh/', 'tired')
hungry_data = load_audio_files('./data/sound_data/dustan_baby/hungry_Nah/', 'hungry')
poop_data = load_audio_files('./data/sound_data/dustan_baby/poop_eairh/', 'poop')
discomfort_data = load_audio_files('./data/sound_data/dustan_baby/uncomfortable_heh/', 'discomfort')

In [18]:
def fast_furior_transform(waveform,sample):
    yf = np.abs(librosa.stft(waveform))
    img = librosa.display.specshow(librosa.amplitude_to_db(yf),y_axis='log')
    plt.colorbar()
    

In [19]:
def Spectrogram(waveform,sample):
    waveform = librosa.stft(waveform)
    waveformdb = librosa.power_to_db(abs(waveform))
    plt.figure(figsize=(15, 3))
    librosa.display.specshow(waveformdb, sr=sample, x_axis='time', y_axis='log',cmap = 'magma')
    plt.colorbar()
    
    return waveformdb

In [20]:
def mel_spectrogram(waveform,sample,n_fft=2560,hop_length=32):
    #fast_fourier_transf = np.fft.fft(waveform)
   
    audio_stft = librosa.feature.melspectrogram(y=waveform,n_fft=n_fft,hop_length=hop_length,fmin = 20,fmax = sample//2)
    log_spectro = librosa.power_to_db(audio_stft)
    librosa.display.specshow(log_spectro, sr=sample, x_axis='time', y_axis='mel',cmap = 'magma')
    
    
    return log_spectro

In [21]:
def rms(waveform,sample):
    # Get RMS value from each frame's magnitude value
    S, phase = librosa.magphase(librosa.stft(waveform))
    rms = librosa.feature.rms(S=S)
    # Plot the RMS energy
    fig, ax = plt.subplots(figsize=(15, 6), nrows=2, sharex=True)
    times = librosa.times_like(rms)
    ax[0].semilogy(times, rms[0], label='RMS Energy')
    ax[0].set(xticks=[])
    ax[0].legend()
    ax[0].label_outer()
    data =librosa.power_to_db(S, ref=np.max)
    librosa.display.specshow(data,y_axis='log', x_axis='time', ax=ax[1],cmap = 'magma')
    ax[1].set(title='log Power spectrogram')
    return data

In [22]:
def zero_crossing(waveform,sample):
    zcrs = librosa.feature.zero_crossing_rate(waveform)
    data = librosa.power_to_db(zcrs)
    print(f"Zero crossing rate: {sum(librosa.zero_crossings(waveform))}")
    plt.figure(figsize=(15, 3))
    plt.plot(zcrs[0])
    plt.title('Action Rock')
    return data


In [23]:
def mel_freq(waveform,sample):
    mfccs = librosa.feature.mfcc(waveform, sr=sample,dct_type = 2,n_mfcc=100)
    data = librosa.power_to_db(mfccs)
    # Displaying  the MFCCs:
    librosa.display.specshow(mfccs, sr=sample, x_axis='mel',cmap = 'magma')
    return data,mfccs

In [26]:
def create_images(dataset,datatype,modeltype,typeimg, label_dir,num):
    # make directory
    # test_directory = f'./data/img_data/{datatype}/{modeltype}/{typeimg}/test/{label_dir}/'
    # train_directory = f'./data/img_data/{datatype}/{modeltype}/{typeimg}/train/{label_dir}/'
    save_directory = f'./data/img_data/{datatype}/{modeltype}/{typeimg}/{label_dir}/'
    # os.makedirs(test_directory, mode=0o777, exist_ok=True)
    # os.makedirs(train_directory, mode=0o777, exist_ok=True)
    os.makedirs(save_directory, mode=0o777, exist_ok=True)
    # for f in os.listdir(save_directory):
    #     os.remove(os.path.join(save_directory, f))
    for i, data in enumerate(dataset):
        sample_rate = data[1]
        waveform = data[0]
        if typeimg == 'Spectrogram':
            spectrogram = Spectrogram(waveform,sample_rate)
        elif typeimg == 'mel_spectrogram':
            spectrogram = mel_spectrogram(waveform,sample_rate)
        elif typeimg == 'rms':
            spectrogram = rms(waveform,sample_rate)
        elif typeimg == 'zero_crossing':
            spectrogram = zero_crossing(waveform,sample_rate)
        elif typeimg == 'mel_freq':
            data_freq,spectrogram = mel_freq(waveform,sample_rate)
        elif typeimg == 'waveform':
            timescale = np.arange(waveform .shape[0])
            fast_fourier_transf = np.fft.fft(waveform)
            plt.plot(timescale, fast_fourier_transf,color = 'k')
        if typeimg != 'waveform':    
            librosa.display.specshow(spectrogram,sr=sample_rate, x_axis='time')
        plt.axis('off')
        # Split test and train images by 30%
        plt.savefig(f'./data/img_data/{datatype}/{modeltype}/{typeimg}/{label_dir}/spec_img{i}{num}.jpg', bbox_inches="tight", pad_inches = 0)
        plt.clf()

In [27]:
create_images(burping_data,modeltype='split_class',datatype = 'dustan',typeimg='mel_freq', label_dir='burpAndPoop',num=0)
create_images(poop_data,modeltype='split_class',datatype = 'dustan',typeimg='mel_freq', label_dir='burpAndPoop',num=1)
create_images(hungry_data,modeltype='split_class',datatype = 'dustan',typeimg='mel_freq',label_dir= 'HungryTiredAndDiscomfort',num=2)
create_images(tired_data,modeltype='split_class',datatype = 'dustan',typeimg='mel_freq',label_dir= 'HungryTiredAndDiscomfort',num=3)
create_images(discomfort_data,modeltype='split_class',datatype = 'dustan',typeimg='mel_freq', label_dir='HungryTiredAndDiscomfort',num=4)

c:\Users\acer\anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Pass y=[-0.0219384  -0.03610521 -0.04121292 ...  0.00415176 -0.00049871
 -0.00027296] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  
c:\Users\acer\anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Pass y=[ 0.00177924 -0.00799471 -0.01225367 ... -0.00759041  0.00428651
  0.00536661] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  
c:\Users\acer\anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Pass y=[-0.00318202 -0.00577083 -0.00303842 ...  0.0100263   0.00637814
  0.00114381] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  
c:\Users\acer\anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Pass y=[-0.01026356  0.00125491  0.00406664 ...  0.00316731  0.00494031
  0.00305476] as keyword args. From versio

<Figure size 432x288 with 0 Axes>

In [12]:
create_images(burping_data,modeltype='burpAndPoop',datatype = 'dustan',typeimg='mel_spectrogram', label_dir='burping',num=2)
create_images(poop_data,modeltype='burpAndPoop',datatype = 'dustan',typeimg='mel_spectrogram', label_dir='poop',num=4)

c:\Users\acer\anaconda3\lib\site-packages\librosa\util\decorators.py:88: UserWarning: n_fft=2560 is too small for input signal of length=2205
  return f(*args, **kwargs)


<Figure size 432x288 with 0 Axes>

In [13]:
create_images(hungry_data,modeltype='HungryTiredAndDiscomfort',datatype = 'dustan',typeimg='mel_spectrogram',label_dir= 'hungry',num=0)
create_images(tired_data,modeltype='HungryTiredAndDiscomfort',datatype = 'dustan',typeimg='mel_spectrogram',label_dir= 'tired',num=1)
create_images(discomfort_data,modeltype='HungryTiredAndDiscomfort',datatype = 'dustan',typeimg='mel_spectrogram', label_dir='discomfort',num=3)

<Figure size 432x288 with 0 Axes>